## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

Create the embedding for this user question:

In [3]:
user_question = "I just discovered the course. Can I still join it?"

What's the first value of the resulting vector?

In [5]:
embedding_model.encode(user_question)[0]

np.float32(0.078222655)

## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [8]:
from tqdm.auto import tqdm

In [9]:
filtered_docs = []

for doc in tqdm(documents):
    if doc['course'] == 'machine-learning-zoomcamp':
        filtered_docs.append(doc)

print(len(filtered_docs))

100%|██████████| 948/948 [00:00<00:00, 2898105.10it/s]

375


## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

* Create a list embeddings
* Iterate over each document
* qa_text = f'{question} {text}'
* compute the embedding for qa_text, append to embeddings
* At the end, let X = np.array(embeddings) (import numpy as np)

What's the shape of X? (X.shape). Include the parantheses.

In [10]:
import numpy as np

In [14]:
embeddings = []

for doc in tqdm(filtered_docs):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    qa_vector = embedding_model.encode(qa_text)
    embeddings.append(qa_vector)

X = np.array(embeddings)
print(X.shape)

100%|██████████| 375/375 [00:37<00:00, 10.11it/s]

(375, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

In [16]:
v = embedding_model.encode(user_question)
scores = X.dot(v)

What's the highest score in the results?

In [17]:
np.max(scores)

np.float32(0.6506573)

## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [18]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

What did you get?

In [34]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [43]:
search_results

[{'text': "✅SOLUTION: pip install confluent-kafka[avro].\nFor some reason, Conda also doesn't include this when installing confluent-kafka via pip.\nMore sources on Anaconda and confluent-kafka issues:\nhttps://github.com/confluentinc/confluent-kafka-python/issues/590\nhttps://github.com/confluentinc/confluent-kafka-python/issues/1221\nhttps://stackoverflow.com/questions/69085157/cannot-import-producer-from-confluent-kafka",
  'section': 'Module 6: streaming with kafka',
  'question': "ModuleNotFoundError: No module named 'avro'",
  'course': 'data-engineering-zoomcamp',
  'id': '1edd4630'},
 {'text': 'GitHub Codespaces offers you computing Linux resources with many pre-installed tools (Docker, Docker Compose, Python).\nYou can also open any GitHub repository in a GitHub Codespace.',
  'section': 'General course-related questions',
  'question': 'Environment - Is GitHub codespaces an alternative to using cli/git bash to ingest the data and create a docker file?',
  'course': 'data-engi

In [44]:
doc_id

'c6a22665'

In [39]:
relevance_total = []

for doc in tqdm(ground_truth):
    v = embedding_model.encode(doc['question'])
    doc_id = doc['document']
    search_results = search_engine.search(v, num_results=5)
    relevance = [result['id'] == doc_id for result in search_results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [00:54<00:00, 33.36it/s]


In [41]:
relevance_total[:5]

[[False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False]]

In [36]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [42]:
relevance_total[0]

[False, False, False, False, False]

In [37]:
print(hit_rate(relevance_total))

0.0


In [28]:
ground_truth[0]['document'] in result_ids

False